In [1]:
# tạo quyền truy cập google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from os import listdir, path
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

raw_folder = '/content/drive/MyDrive/Data/Datasheet/'

def save_data(raw_folder=raw_folder):
    dest_size = (300, 300)
    print("Bắt đầu xử lý ảnh.....")

    pixels = []
    labels = []

    for folder_name in listdir(raw_folder):
        folder_path = path.join(raw_folder, folder_name)
        if path.isdir(folder_path) and folder_name != '.DS_Store': # Check if it's a directory
            print('Folder = ', folder_name)
            for file_name in listdir(folder_path):
                if file_name != '.DS_Store':
                    img_path = path.join(folder_path, file_name)
                    print('File = ', file_name)
                    img = cv2.imread(img_path)
                    if img is not None: # Check if image was loaded successfully
                        pixels.append(
                            cv2.resize(
                                img,
                                dsize=(300, 300)
                            )
                        )
                        labels.append(folder_name)
                    else:
                        print(f"Warning: Could not read image file: {img_path}. Skipping...")
        elif not path.isdir(folder_path):
            print(f"Skipping non-directory item: {folder_name}")

    pixels = np.array(pixels)
    labels = np.array(labels)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    print(labels)

    file = open('pix.data', 'wb')
    pickle.dump((pixels, labels), file)
    file.close()

    return

def load_data():
    file = open('pix.data', 'rb')
    (pixels, labels) = pickle.load(file)
    file.close()

    print(pixels.shape)
    print(labels.shape)

    return pixels, labels

save_data()
X, Y = load_data()

Bắt đầu xử lý ảnh.....
Folder =  Conor
File =  20.png
File =  conor15.png
File =  conor3.jpg
File =  conor6.png
File =  conor2.jpg
File =  conor19.png
File =  conor17.png
File =  conor5.png
File =  conor16.png
File =  conor10.png
File =  conor13.png
File =  conor9.png
File =  conor7.png
File =  conor11.png
File =  conor12.png
Folder =  Faker
File =  faker1.jpg
File =  faker2.jpg
File =  faker4.jpg
File =  faker5.jpg
File =  faker7.jpg
File =  faker8.jpg
File =  faker10.jpg
File =  faker12.jpg
File =  faker13.jpg
File =  faker14.jpg
File =  faker16.jpg
File =  faker17.jpg
File =  faker18.jpg
File =  faker19.jpg
File =  faker20.jpg
Folder =  SonTung
File =  1.jpg
File =  2.jpg
File =  4.jpg
File =  5.jpg
File =  6.jpg
File =  7.jpg
File =  10.jpg
File =  9.jpg
File =  11.jpg
File =  12.jpg
File =  13.jpg
File =  14.jpg
File =  16.jpg
File =  17.jpg
File =  19.jpg
Folder =  Test
File =  Conor
File =  SonTung
File =  Faker
Skipping non-directory item: vggmodel.h5
[[1 0 0]
 [1 0 0]
 [1 0 0]

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

print(X_train.shape)
print(Y_train.shape)

def get_model():
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    input = Input(shape=(300, 300, 3), name='image_input')
    output_vgg16_conv = model_vgg16_conv(input)

    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(3, activation='softmax', name='predictions')(x)

    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return my_model

vggmodel = get_model()

filepath = "/content/drive/MyDrive/Data/Datasheet/weights-{epoch:02d}-{val_accuracy:.2f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
                         rescale=1./255,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         brightness_range=[0.2, 1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1. / 255)

vgghist=vggmodel.fit(aug.flow(X_train, Y_train, batch_size=64),
                                epochs=50,
                                validation_data=aug.flow(X_test, Y_test,
                                batch_size=len(X_test)),
                                callbacks=callbacks_list)

vggmodel.save('/content/drive/MyDrive/Data/Datasheet/vggmodel.h5')

(31, 300, 300, 3)
(31, 3)
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38s/step - accuracy: 0.4839 - loss: 1.2251
Epoch 1: val_accuracy improved from -inf to 0.35714, saving model to /content/drive/MyDrive/Data/Datasheet/weights-01-0.36.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 83s 83s/step - accuracy: 0.4839 - loss: 1.2251 - val_accuracy: 0.3571 - val_loss: 21.0249
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49s/step - accuracy: 0.3226 - loss: 25.2715
Epoch 2: val_accuracy improved from 0.35714 to 0.42857, saving model to /content/drive/MyDrive/Data/Datasheet/weights-02-0.43.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 91s 91s/step - accuracy: 0.3226 - loss: 25.2715 - val_accuracy: 0.4286 - val_loss: 91.2621
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46s/step - accuracy: 0.3548 - loss: 93.3548
Epoch 3: val_accuracy did not improve from 0.42857
1/1 ━━━━━━━━━━━━━━━━━━━━ 60s 60s/step - accuracy: 0.3548 - loss: 93.3548 - val_accuracy: 0.2143 - val_loss: 199.3120
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33s/step - accuracy: 0.3871 - loss: 160.7811
Epoch 4: